In [1]:
import polars as pl

from rich.pretty import pprint

from neo_builder.db import Neo4jConnector
from neo_builder.build import GraphBuilder, GraphBuildMap

from neo_builder.graphs.ml_small.rels import MovieGenreRelBuilder
from neo_builder.graphs.ml_small.nodes import MovieNodeBuilder, GenreNodeBuilder

In [2]:
def get_data_items(csv_path: str) -> list[dict]:
    df = pl.read_csv(source=csv_path)
    return df.to_dicts()

In [ ]:
movie_data_items = get_data_items(
    csv_path="/resources/datasets/ml-latest-small/movies.csv"
)

print(len(movie_data_items))
pprint(movie_data_items[:5])

In [4]:
constraints = [
    {
        "constraint_name": "movie_id",
        "node_type": "Movie",
        "node_property": "node_id",
    },
    {
        "constraint_name": "genre_id",
        "node_type": "Genre",
        "node_property": "node_id",
    },
]

In [5]:
neo_connector = Neo4jConnector()
build_map = GraphBuildMap(
    **{
        "node_builders": [
            MovieNodeBuilder(
                neo_connector=neo_connector,
                data_items=movie_data_items,
                build_from_csv=True,
            ),
            GenreNodeBuilder(
                neo_connector=neo_connector,
                data_items=movie_data_items,
                build_from_csv=True,
            ),
        ],
        "rel_builders": [
            MovieGenreRelBuilder(
                neo_connector,
                data_items=movie_data_items,
                build_from_csv=True,
            ),
        ],
    }
)


In [ ]:
builder = GraphBuilder(
    neo_connector=neo_connector,
    build_map=build_map,
    constraints=constraints,
)

In [ ]:
builder.build()